# Import Packages

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import quantities as pq
import elephant
import spm1d
import neurotic
from neurotic.gui.config import _neo_epoch_to_dataframe

pq.mN = pq.UnitQuantity('millinewton', pq.N/1e3, symbol = 'mN');  # define millinewton

# IPython Magics

In [ ]:
# make figures interactive and open in a separate window
# %matplotlib qt

# make figures interactive and inline
%matplotlib notebook

# make figures non-interactive and inline
# %matplotlib inline

# Data Parameters

In [ ]:
# specify the data sets to analyze
data_sets = [
    'IN VIVO / JG08 / 2018-06-21 / 002',
    'IN VIVO / JG08 / 2018-06-24 / 001',
]

# load the metadata containing file paths
metadata = neurotic.MetadataSelector(file='../../data/metadata.yml')

# store metadata in a dictionary that we will add to later
data = {}
for data_set_name in data_sets:
    metadata.select(data_set_name)
    data[data_set_name] = {}
    data[data_set_name]['metadata'] = metadata.selected_metadata

In [ ]:
# select which swallow sequences to use

data['IN VIVO / JG08 / 2018-06-21 / 002']['time_windows_to_keep'] = [
#     [-np.inf, np.inf], # keep everything
    [659, 726.1], # tension maximized and no perturbation
#     [666.95, 726.1], # tension maximized and no perturbation, and extra long large hump excluded
]

data['IN VIVO / JG08 / 2018-06-24 / 001']['time_windows_to_keep'] = [
#     [-np.inf, np.inf], # keep everything
    [2244.7, 2259.9], [2269.5, 2355.95], # tension maximized and no perturbation
#     [2244.7, 2259.9], [2269.5, 2290.2], [2307, 2355.95], # tension maximized and no perturbation, and extra long large hump excluded
]

# Import and Process the Data

In [ ]:
for data_set_name, d in data.items():

    # read in the data
    blk = neurotic.load_dataset(d['metadata'])
    signalNameToIndex = {sig.name:i for i, sig in enumerate(blk.segments[0].analogsignals)}

    # grab the force vs time data and rescale to mN
    d['force_sig'] = blk.segments[0].analogsignals[signalNameToIndex['Force']].rescale('mN')

    # apply a super-low-pass filter to force signal
    d['smoothed_force_sig'] = elephant.signal_processing.butter(  # may raise a FutureWarning
        signal = d['force_sig'],
        lowpass_freq = 0.5*pq.Hz,
    )

    # keep only epochs that are entirely inside the time windows
    epochs_df = _neo_epoch_to_dataframe(blk.segments[0].epochs)
    epochs_df = epochs_df[np.any(list(map(lambda t: (t[0] <= epochs_df['Start (s)']) & (epochs_df['End (s)'] <= t[1]), d['time_windows_to_keep'])), axis=0)]

    # copy middle times (end of large hump and start of small hump) into 'force' epochs
    for i, epoch in epochs_df[epochs_df['Type'] == 'force'].iterrows():
        for j, subepoch in epochs_df[epochs_df['Type'] == 'large hump'].iterrows():
            if subepoch['Start (s)'] >= epoch['Start (s)']-1e-7 and subepoch['End (s)'] <= epoch['End (s)']+1e-7:
                epochs_df.loc[i, 'Middle (s)'] = subepoch['End (s)']

    # drop all but 'force' rows
    epochs_df = epochs_df[epochs_df['Type'] == 'force']

    # find max forces in each epoch
    for i, epoch in epochs_df.iterrows():
        epochs_df.loc[i,          'large max'] = max(         d['force_sig'].time_slice(epoch['Start (s)'] *pq.s, epoch['Middle (s)']*pq.s).magnitude)[0]
        epochs_df.loc[i,          'small max'] = max(         d['force_sig'].time_slice(epoch['Middle (s)']*pq.s, epoch['End (s)']   *pq.s).magnitude)[0]
        epochs_df.loc[i, 'smoothed large max'] = max(d['smoothed_force_sig'].time_slice(epoch['Start (s)'] *pq.s, epoch['Middle (s)']*pq.s).magnitude)[0]
        epochs_df.loc[i, 'smoothed small max'] = max(d['smoothed_force_sig'].time_slice(epoch['Middle (s)']*pq.s, epoch['End (s)']   *pq.s).magnitude)[0]

    # colors
    epochs_df = epochs_df.assign(colormap_arg = np.linspace(0, 1, len(epochs_df)))

    d['epochs_df'] = epochs_df

# Plots

In [ ]:
# color map
cm = plt.cm.cool
# cm = plt.cm.brg
# cm = plt.cm.RdBu

sns.set(
#     context = 'poster',
    style = 'ticks',
    font_scale = 1,
    font = 'Palatino Linotype',
)

# fig size in inches
figsize = (9,4)

##### Figure 1: Plot forces across real time

In [ ]:
plt.figure(1, figsize=figsize)
for i, data_set_name in enumerate(data_sets):
    d = data[data_set_name]
    plt.subplot(1, len(data), i+1)
    plt.title('{}\nt = {}'.format(data_set_name, d['time_windows_to_keep']))
    plt.ylabel('Force (mN)')
    plt.xlabel('Original chart time (s)')
    for j, epoch in d['epochs_df'].iterrows():
        epoch_force_sig = d['force_sig'].time_slice(epoch['Start (s)']*pq.s, epoch['End (s)']*pq.s)
        plt.plot(epoch_force_sig.times, epoch_force_sig.as_array(), color=cm(epoch['colormap_arg']))
    sns.despine(ax=plt.gca(), offset=10, trim=True) # offset axes from plot
plt.tight_layout()

##### Figure 2: Plot forces with aligned start times

In [ ]:
plt.figure(2, figsize=figsize)
for i, data_set_name in enumerate(data_sets):
    d = data[data_set_name]
    plt.subplot(1, len(data), i+1)
    plt.title('{}\nt = {}'.format(data_set_name, d['time_windows_to_keep']))
    plt.ylabel('Force (mN)')
    plt.xlabel('Zeroed time (s)')
    for j, epoch in d['epochs_df'].iterrows():
        epoch_force_sig = d['force_sig'].time_slice(epoch['Start (s)']*pq.s, epoch['End (s)']*pq.s)
        epoch_force_sig.t_start = 0*pq.s
        plt.plot(epoch_force_sig.times, epoch_force_sig.as_array(), color=cm(epoch['colormap_arg']))
    sns.despine(ax=plt.gca(), offset=10, trim=True) # offset axes from plot
plt.tight_layout()

##### Figure 3: Plot forces with uniformaly normalized time

In [ ]:
plt.figure(3, figsize=figsize)
for i, data_set_name in enumerate(data_sets):
    d = data[data_set_name]
    plt.subplot(1, len(data), i+1)
    plt.title('{}\nt = {}'.format(data_set_name, d['time_windows_to_keep']))
    plt.ylabel('Force (mN)')
    plt.xlabel('Uniformly normalized time')
    for j, epoch in d['epochs_df'].iterrows():
        epoch_force_sig = d['force_sig'].time_slice(epoch['Start (s)']*pq.s, epoch['End (s)']*pq.s)
        epoch_force_sig.t_start = 0*pq.s
        epoch_force_sig.sampling_period = 1/len(epoch_force_sig.times)*pq.s
        plt.plot(epoch_force_sig.times, epoch_force_sig.as_array(), color=cm(epoch['colormap_arg']))
    sns.despine(ax=plt.gca(), offset=10, trim=True) # offset axes from plot
plt.tight_layout()

##### Figure 4: Plot normalized forces with uniformaly normalized time

In [ ]:
plt.figure(4, figsize=figsize)
for i, data_set_name in enumerate(data_sets):
    d = data[data_set_name]
    plt.subplot(1, len(data), i+1)
    plt.title('{}\nt = {}'.format(data_set_name, d['time_windows_to_keep']))
    plt.ylabel('Normalized force')
    plt.xlabel('Uniformly normalized time')
    for j, epoch in d['epochs_df'].iterrows():
        epoch_force_sig = d['force_sig'].time_slice(epoch['Start (s)']*pq.s, epoch['End (s)']*pq.s) / epoch['smoothed large max']
        epoch_force_sig.t_start = 0*pq.s
        epoch_force_sig.sampling_period = 1/len(epoch_force_sig.times)*pq.s
        plt.plot(epoch_force_sig.times, epoch_force_sig.as_array(), color=cm(epoch['colormap_arg']))
    sns.despine(ax=plt.gca(), offset=10, trim=True) # offset axes from plot
plt.tight_layout()

##### Figure 5: Plot forces with time normalized separately for large and small humps

In [ ]:
plt.figure(5, figsize=figsize)
for i, data_set_name in enumerate(data_sets):
    d = data[data_set_name]
    plt.subplot(1, len(data), i+1)
    plt.title('{}\nt = {}'.format(data_set_name, d['time_windows_to_keep']))
    plt.ylabel('Force (mN)')
    plt.xlabel('Phase-dependent normalized time')
    for j, epoch in d['epochs_df'].iterrows():
        epoch_large_force_sig = d['force_sig'].time_slice(epoch['Start (s)']*pq.s, epoch['Middle (s)']*pq.s)
        epoch_small_force_sig = d['force_sig'].time_slice(epoch['Middle (s)']*pq.s, epoch['End (s)']*pq.s)
        epoch_large_force_sig.t_start = 0*pq.s
        epoch_small_force_sig.t_start = 1*pq.s
        epoch_large_force_sig.sampling_period = 1/len(epoch_large_force_sig.times)*pq.s
        epoch_small_force_sig.sampling_period = 1/len(epoch_small_force_sig.times)*pq.s
        plt.plot(np.concatenate([epoch_large_force_sig.times, epoch_small_force_sig.times]), np.concatenate([epoch_large_force_sig.as_array(), epoch_small_force_sig.as_array()]), color=cm(epoch['colormap_arg']))
    sns.despine(ax=plt.gca(), offset=10, trim=True) # offset axes from plot
plt.tight_layout()

##### Figure 6: Plot normalized forces with time normalized separately for large and small humps

In [ ]:
plt.figure(6, figsize=figsize)
for i, data_set_name in enumerate(data_sets):
    d = data[data_set_name]
    plt.subplot(1, len(data), i+1)
    plt.title('{}\nt = {}'.format(data_set_name, d['time_windows_to_keep']))
    plt.ylabel('Normalized force')
    plt.xlabel('Phase-dependent normalized time')
    for j, epoch in d['epochs_df'].iterrows():
        epoch_large_force_sig = d['force_sig'].time_slice(epoch['Start (s)']*pq.s, epoch['Middle (s)']*pq.s) / epoch['smoothed large max']
        epoch_small_force_sig = d['force_sig'].time_slice(epoch['Middle (s)']*pq.s, epoch['End (s)']*pq.s)  / epoch['smoothed large max']
        epoch_large_force_sig.t_start = 0*pq.s
        epoch_small_force_sig.t_start = 1*pq.s
        epoch_large_force_sig.sampling_period = 1/len(epoch_large_force_sig.times)*pq.s
        epoch_small_force_sig.sampling_period = 1/len(epoch_small_force_sig.times)*pq.s
        plt.plot(np.concatenate([epoch_large_force_sig.times, epoch_small_force_sig.times]), np.concatenate([epoch_large_force_sig.as_array(), epoch_small_force_sig.as_array()]), color=cm(epoch['colormap_arg']))
    sns.despine(ax=plt.gca(), offset=10, trim=True) # offset axes from plot
plt.tight_layout()

##### Figure 7: Reproduce Fig. 5 using interpolated data

In [ ]:
# interpolate and resample the signals at regular intervals in preparation for averaging
for data_set_name, d in data.items():

    epoch_large_force_values = []
    epoch_small_force_values = []
    for i, epoch in d['epochs_df'].iterrows():
        epoch_large_force_sig = d['force_sig'].time_slice(epoch['Start (s)']*pq.s, epoch['Middle (s)']*pq.s)
        epoch_small_force_sig = d['force_sig'].time_slice(epoch['Middle (s)']*pq.s, epoch['End (s)']*pq.s)
        epoch_large_force_values.append(epoch_large_force_sig.as_array().flatten())
        epoch_small_force_values.append(epoch_small_force_sig.as_array().flatten())

    # number of points per curve before interpolation
#     print([x.shape for x in epoch_large_force_values])
#     print([x.shape for x in epoch_small_force_values])

    # linear interpolation and resampling to n_samples data points
    n_samples = 1000
    epoch_large_force_values = spm1d.util.interp(epoch_large_force_values, Q=n_samples)
    epoch_small_force_values = spm1d.util.interp(epoch_small_force_values, Q=n_samples)

    # number of points per curve after interpolation
#     print([x.shape for x in epoch_large_force_values])
#     print([x.shape for x in epoch_small_force_values])

    # combine large and small into unified time series
    epoch_all_force_values = np.concatenate([epoch_large_force_values, epoch_small_force_values], axis=1)
#     print(epoch_all_force_values.shape)

    # evenly spaced time values from 0 to 2
    times = np.linspace(0, 2, 2*n_samples)
    
    d['resampled_epoch_all_force_values'] = epoch_all_force_values
    d['resampled_times'] = times

In [ ]:
plt.figure(7, figsize=figsize)
for i, data_set_name in enumerate(data_sets):
    d = data[data_set_name]
    plt.subplot(1, len(data), i+1)
    plt.title('{}\nt = {}'.format(data_set_name, d['time_windows_to_keep']))
    plt.ylabel('Force (mN)')
    plt.xlabel('Phase-dependent normalized time')
    for epoch, colormap_arg in zip(d['resampled_epoch_all_force_values'], d['epochs_df']['colormap_arg']):
        plt.plot(d['resampled_times'], epoch, color=cm(colormap_arg))
    sns.despine(ax=plt.gca(), offset=10, trim=True) # offset axes from plot
plt.tight_layout()

##### Figure 8: Plot mean and standard deviation cloud

In [ ]:
plt.figure(8, figsize=figsize)
for i, data_set_name in enumerate(data_sets):
    d = data[data_set_name]
    plt.subplot(1, len(data), i+1)
    plt.title('{}\nt = {}'.format(data_set_name, d['time_windows_to_keep']))
    plt.ylabel('Force (mN)')
    plt.xlabel('Phase-dependent normalized time')
    spm1d.plot.plot_mean_sd(d['resampled_epoch_all_force_values'])#, x=d['resampled_times'])

#     # verify that spm1d.plot.plot_mean_sd does what I think it does
#     mean = np.mean(d['resampled_epoch_all_force_values'], axis=0)
#     std = np.std(d['resampled_epoch_all_force_values'], axis=0)
#     plt.plot(mean, 'w:')
#     plt.plot(mean+std, 'r:')
#     plt.plot(mean-std, 'b:')

    sns.despine(ax=plt.gca(), offset=10, trim=True) # offset axes from plot
plt.tight_layout()

##### Figure 9: Overlay mean and standard deviation cloud

In [ ]:
plt.figure(9, figsize=(9,6))
for i, data_set_name in enumerate(data_sets):
    d = data[data_set_name]
    plt.ylabel('Force (mN)')
    plt.xlabel('Phase-dependent normalized time')
    spm1d.plot.plot_mean_sd(d['resampled_epoch_all_force_values'])#, x=d['resampled_times'])
    plt.xlim([0, 2000])
    plt.ylim([0, 320])
    
#     # verify that spm1d.plot.plot_mean_sd does what I think it does
#     mean = np.mean(d['resampled_epoch_all_force_values'], axis=0)
#     std = np.std(d['resampled_epoch_all_force_values'], axis=0)
#     plt.plot(mean, 'w:')
#     plt.plot(mean+std, 'r:')
#     plt.plot(mean-std, 'b:')

    sns.despine(ax=plt.gca(), offset=10, trim=True) # offset axes from plot
plt.tight_layout()